In [1]:
import pandas as pd
import numpy as np
import pathlib

%load_ext lab_black

# Creating an aggregate burden index

Although we will not be using a aggregate burden index for v1.0 of the CEJST, the USDS team wanted to demonstrate how even duplicating CalEnviroScreen's cumulative index (or a loose interpretation of it) would impact the communities highlighted. Here, the scoring procedure that we use is lifted as closely as possible from CalEnviroScreen, including the categorization of burdens and the weighting between cumulative score. 

The data team believes that a threshold methodology has significant limitations that an aggregate or cumulative burden index could remediate, and presents the following as an example of such an index. 

In [2]:
SCORE_DIR = pathlib.Path.cwd().parent / "data" / "score" / "csv" / "full"

In [3]:
usa = pd.read_csv(
    SCORE_DIR / "usa.csv",
    dtype={"GEOID10_TRACT": str},
)

/usr/local/Cellar/jupyterlab/3.2.8/libexec/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (1,2,3,4,75,76,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
## environment
toxins_category = [
    "Percent pre-1960s housing (lead paint indicator) (percentile)",
    "Proximity to Risk Management Plan (RMP) facilities (percentile)",
    "Proximity to NPL sites (percentile)",
    "Proximity to hazardous waste sites (percentile)",
    "Wastewater discharge (percentile)",
]

## sensitive populations
health_category = [
    "Diagnosed diabetes among adults aged greater than or equal to 18 years (percentile)",
    "Current asthma among adults aged greater than or equal to 18 years (percentile)",
    "Coronary heart disease among adults aged greater than or equal to 18 years (percentile)",
    "Low life expectancy (percentile)",
]

## exposure
built_environment_category = [
    "Expected building loss rate (Natural Hazards Risk Index) (percentile)",
    "Expected agricultural loss rate (Natural Hazards Risk Index) (percentile)",
    "Expected population loss rate (Natural Hazards Risk Index) (percentile)",
    "Energy burden (percentile)",
    "Diesel particulate matter exposure (percentile)",
    "Traffic proximity and volume (percentile)",
    "PM2.5 in the air (percentile)",
]

## socioeconomic
socioeconomic_category = [
    "Unemployment (percent) (percentile)",
    "Housing burden (percent) (percentile)",
    "Low median household income as a percent of area median income (percentile)",
    "Percent of households in linguistic isolation (percentile)",
    "Percent of individuals below 200% Federal Poverty Line, imputed and adjusted (percentile)",
    "Percent individuals age 25 or over with less than high school degree (percentile)",
    "Percent of individuals < 100% Federal Poverty Line (percentile)",
]

In [5]:
usa["toxins_cat"] = usa[toxins_category].mean(axis=1)
usa["built_env_cat"] = usa[built_environment_category].mean(axis=1)
usa["health_cat"] = usa[health_category].mean(axis=1)
usa["ses_cat"] = usa[socioeconomic_category].mean(axis=1)


usa["pollution_burden"] = 0.5 * usa["toxins_cat"] + usa["built_env_cat"]
usa["population_characteristics"] = usa["health_cat"] + usa["ses_cat"]
poll_max = usa["pollution_burden"].max()
pop_max = usa["population_characteristics"].max()

usa["scaled_pollution_burden"] = usa["pollution_burden"] / poll_max
usa["scaled_population_characteristics"] = usa["population_characteristics"] / pop_max

usa["cal_score"] = (
    usa["scaled_pollution_burden"] * usa["scaled_population_characteristics"]
)
usa["pct_cal_score"] = usa["cal_score"].rank(pct=True)

In [6]:
# this shows the number of communities identified by Score N (base)
# that are also identified by our cumulative burden metric (at some threshold)

for cutoff in [0.65, 0.8, 0.825, 0.85, 0.9]:

    usa["pct_cal_score_" + str(cutoff)] = usa["pct_cal_score"] >= cutoff
    display(
        pd.crosstab(
            usa["pct_cal_score_" + str(cutoff)],
            usa["Definition N (communities)"],
        )
    )
    display(
        pd.crosstab(
            usa["pct_cal_score_" + str(cutoff)],
            usa["Definition N (communities)"],
            normalize=True,
        )
    )

Definition N (communities),False,True
pct_cal_score_0.65,,
False,42039,6823
True,5423,19849


Definition N (communities),False,True
pct_cal_score_0.65,,
False,0.567068,0.092036
True,0.073151,0.267745


Definition N (communities),False,True
pct_cal_score_0.8,,
False,46152,13540
True,1310,13132


Definition N (communities),False,True
pct_cal_score_0.8,,
False,0.622548,0.182642
True,0.017671,0.177139


Definition N (communities),False,True
pct_cal_score_0.825,,
False,46575,14923
True,887,11749


Definition N (communities),False,True
pct_cal_score_0.825,,
False,0.628254,0.201298
True,0.011965,0.158483


Definition N (communities),False,True
pct_cal_score_0.85,,
False,46898,16405
True,564,10267


Definition N (communities),False,True
pct_cal_score_0.85,,
False,0.632611,0.221288
True,0.007608,0.138492


Definition N (communities),False,True
pct_cal_score_0.9,,
False,47288,19625
True,174,7047


Definition N (communities),False,True
pct_cal_score_0.9,,
False,0.637872,0.264723
True,0.002347,0.095058


## Does it square with calenvironscreen? 

Here we compare OUR work with the data from CalEnviroScreen, limiting to California. We see reasonable agreement -- the vast majority (>90%) of tracts at or above 90th percentile match.

In [7]:
DATA_DIR = pathlib.Path.cwd().parent / "data" / "dataset"

true_ces = pd.read_csv(
    DATA_DIR / "calenviroscreen4/data06.csv",
    dtype={"GEOID10_TRACT": str},
)

In [8]:
ces_merged = usa.merge(true_ces, on="GEOID10_TRACT", how="right")

In [9]:
ces_merged["new_cal_score"] = ces_merged["pct_cal_score"].rank(pct=True)
ces_merged["new_cal_flag"] = ces_merged["new_cal_score"] >= 0.9

In [10]:
ces_merged["new_cal_flag"].value_counts(normalize=True)

False    0.900311
True     0.099689
Name: new_cal_flag, dtype: float64

In [11]:
ces_merged["any_flag"] = (
    ces_merged["pct_cal_score_0.9"] | ces_merged["Definition N (communities)"]
)

In [12]:
# Each row of the following outputs answers the question:
# what share of communities at or above the 90th percentile for our version of aggregate burden fall into
# each percentile range for the true calenviroscreen? For example, 38% of tracts that are above 90th percentile
# for our metric are in the highest score bucket on CES.

ces_merged.groupby("pct_cal_score_0.9")["DRAFT CES 4.0\nPercentile Range"].value_counts(
    dropna=False, normalize=True
)

pct_cal_score_0.9  DRAFT CES 4.0\nPercentile Range
False              10-15%                             0.052527
                   20-25%                             0.052527
                   40-45%                             0.052527
                   1-5% (lowest scores)               0.052395
                   15-20%                             0.052395
                   25-30%                             0.052395
                   30-35%                             0.052395
                   35-40%                             0.052395
                   5-10%                              0.052395
                   45-50%                             0.052263
                   50-55%                             0.052263
                   60-65%                             0.052130
                   55-60%                             0.051998
                   65-70%                             0.051336
                   70-75%                             0.050410
    

In [13]:
# Easier to read!
ces_merged.groupby("pct_cal_score_0.9")["DRAFT CES 4.0\nPercentile Range"].value_counts(
    dropna=False, normalize=True
).rename("share").reset_index().pivot_table(
    index="pct_cal_score_0.9", columns="DRAFT CES 4.0\nPercentile Range", values="share"
)

DRAFT CES 4.0 Percentile Range,1-5% (lowest scores),10-15%,15-20%,20-25%,25-30%,30-35%,35-40%,40-45%,45-50%,5-10%,50-55%,55-60%,60-65%,65-70%,70-75%,75-80%,80-85%,85-90%,90-95%,95-100% (highest scores)
pct_cal_score_0.9,,,,,,,,,,,,,,,,,,,,
False,0.052395,0.052527,0.052395,0.052527,0.052395,0.052395,0.052395,0.052527,0.052263,0.052395,0.052263,0.051998,0.052130,0.051336,0.050410,0.04869,0.047499,0.041413,0.037841,0.028844
True,NaN,NaN,NaN,NaN,NaN,0.002096,NaN,NaN,0.002096,NaN,0.004193,0.006289,0.006289,0.016771,0.033543,0.05870,0.079665,0.174004,0.232704,0.375262


In [14]:
# Any flag here is score N or our version of cumulative burden. This table can be read the same way as above

ces_merged.groupby("any_flag")["DRAFT CES 4.0\nPercentile Range"].value_counts(
    dropna=False, normalize=True
).rename("share").reset_index().pivot_table(
    index="any_flag", columns="DRAFT CES 4.0\nPercentile Range", values="share"
).T

any_flag,False,True
DRAFT CES 4.0 Percentile Range,,
1-5% (lowest scores),0.079518,NaN
10-15%,0.077108,0.004255
15-20%,0.075904,0.005892
20-25%,0.072691,0.011457
25-30%,0.071285,0.013421
30-35%,0.068876,0.017676
35-40%,0.066466,0.021277
40-45%,0.061847,0.029133
45-50%,0.056225,0.037971
